# Generating line geometries from a table of line-events with known Route IDs and mile markers

Suppose we have a table of line events with Route ID and mile marker information about where each event happened, but with no associated geometries. 
Suppose further that we also have the linework of the reference routes. 

We can use `linref` to generate geometries for each of the rows of the line data. The example below shows the step-by-step process.

In [8]:
# Import dependencies
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import linref as lr

First, let's create the sample DataFrames and GeoDataFrames we'll be using throughout this example.

In [9]:
# `line_df` is the DataFrame containing the lines with route ID and mile marker info but without any
# actual geometries 
line_df = pd.DataFrame({
    'line_id':[1,2,3,4],
    'route_id':['A','A','B','B'],
    'beg':[1,2,2,3],
    'end':[2,3,3,4]
})


# `ref_gdf` is the GeoDataFrame that contains the reference linework of the roadway network
ref_gdf = gpd.GeoDataFrame({
    'route_id': ['A','B'],
    'beg': [0,0],
    'end': [5,5],
    'geometry': [LineString(((0,0),(5,0))), LineString(((0,2),(5,2)))]
})

The first processing step is to create `EventCollection` objects for both sets:

In [10]:
# For the reference object, notice how we specify the `beg` and `end` parameters because we are dealing with a link-based object.
# The idea here is that the `beg` and `end` columns indicate which columns contain the start and end mile marker information for 
# each link. 
ref_ec = lr.EventsCollection(
    ref_gdf,
    keys=['route_id'],
    beg='beg',
    end='end',
    geom='geometry'
)

# For the line object, we also need to specify the `beg` and `end` parameters. 
line_ec = lr.EventsCollection(
    line_df, 
    keys=['route_id'],
    beg='beg',
    end='end'
)

We then need to build the routes for the reference `EventsCollection` object.
This step directs the `linref` library to perform some internal processing steps that are required for GIS-based operations.

In [11]:
ref_ec.build_routes()

Then, we can create the new geometries by merging the `line_ec` with the `ref_ec` and applying the `cut()` method. 
This will generate a numpy array of `MultiLineString` geometries, which can easily be added to your DataFrame. 

In [12]:
# Now we can merge the point and reference EventsCollections and cut new geometries. The `merge` method defines 
# a linearly referenced relationship between the target and merged collections which can be used to cut new geometries. 
# This relationship is captured and managed within an `EventsMerge` object, commonly referred to as `em`.
em = line_ec.merge(ref_ec)

# The `cut` method is one of the many aggregators available using the `EventsMerge` class.
new_geoms = em.cut()
print(new_geoms)

2    MULTILINESTRING ((2 2, 3 2))
3    MULTILINESTRING ((3 2, 4 2))
0    MULTILINESTRING ((1 0, 2 0))
1    MULTILINESTRING ((2 0, 3 0))
Name: route, dtype: object


Finally, if you want to want to create a full-fledged GeoDataFrame, you can just make a copy of your original input DataFrame and add the new array of lines as the geometry column.

In [14]:
# Create a new GeoDataFrame with the interpolated geometries
line_gdf = gpd.GeoDataFrame(line_df, geometry=new_geoms)
print(line_gdf)

   line_id route_id  beg  end  \
0        1        A    1    2   
1        2        A    2    3   
2        3        B    2    3   
3        4        B    3    4   

                                            geometry  
0  MULTILINESTRING ((1.00000 0.00000, 2.00000 0.0...  
1  MULTILINESTRING ((2.00000 0.00000, 3.00000 0.0...  
2  MULTILINESTRING ((2.00000 2.00000, 3.00000 2.0...  
3  MULTILINESTRING ((3.00000 2.00000, 4.00000 2.0...  
   line_id route_id  beg  end
0        1        A    1    2
1        2        A    2    3
2        3        B    2    3
3        4        B    3    4
